In [ ]:
import flopy as fp
import flopy.utils as fu
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [ ]:
modelpath = Path('../data/voronoi/')

In [ ]:
base_sim = fp.mf6.MFSimulation.load(sim_ws=str(modelpath))

In [ ]:
base_m = base_sim.get_model()

In [ ]:
sfr_base_obs = pd.read_csv(modelpath / 'sfr_obs.csv', index_col=0)

In [ ]:
sfr_base_obs = sfr_base_obs[['GAGE1','GAGE2']]

In [ ]:
sfr_base_obs

## find all the locations of cells

In [ ]:
cells = pd.DataFrame.from_records(base_m.dis.cell2d.array)
cells

# find the locations of cells that  contain streams and let's not place wells in or under them 

In [ ]:
sfrcells = pd.DataFrame.from_records(base_m.sfr.packagedata.array)
sfrcells

In [ ]:
# make a new columns with just the cellid regardless of layer
sfrcells['cell_no_layer'] = [i[1] for i in sfrcells.cellid]
sfrcells

## now we can find the cells from disv that are _not_ also in SFR using a `set` operation

In [ ]:
len(set(cells.icell2d)), len(set(sfrcells.cell_no_layer)), len(set(cells.icell2d))-len(set(sfrcells.cell_no_layer))

In [ ]:
cells_for_wells = list(set(cells.icell2d)-set(sfrcells.cell_no_layer))
len(cells_for_wells)

### now lets load the well file to find a representative pumping rate

In [ ]:
wells = pd.DataFrame.from_records(base_m.wel.stress_period_data.array[0])
wells

In [ ]:
newq = wells.q.mean()
newq

## now let's see about adding a new well

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws=str(modelpath))

In [ ]:
testmod = sim.get_model()

In [ ]:
wells.dtypes

In [ ]:
twell = fp.mf6.ModflowGwfwel(testmod, save_flows=True, stress_period_data=[[(0,0), newq]])

In [ ]:
runpath = Path('../data/depletion/')
sim.set_sim_path(str(runpath))

In [ ]:
sim.exe_name='mf6'
sim.write_simulation()

In [ ]:
sim.run_simulation()

In [ ]:
testmod.output.list().get_data()

In [ ]:
sfr_test = pd.read_csv(runpath / 'sfr_obs.csv')[['GAGE1','GAGE2']]

In [ ]:
sfr_test

In [ ]:
sfr_base_obs

In [ ]:
(sfr_base_obs.GAGE2.values-sfr_test.GAGE2.values)

## now let's make a dataframe to hold the results

In [ ]:
cellids = np.hstack((cells.icell2d.values, cells.icell2d.values, cells.icell2d.values)) # three layers
layers = np.hstack(([0 for i in range(len(cells.icell2d.values))],
                   [1 for i in range(len(cells.icell2d.values))],
                   [2 for i in range(len(cells.icell2d.values))]))

In [ ]:
layers

In [ ]:
cells_with_layers = list(zip(layers,cellids))

In [ ]:
depletion_results = pd.DataFrame(index=pd.MultiIndex.from_tuples(cells_with_layers), data = {'Gage1':np.nan, 'Gage2':np.nan})
depletion_results

# now make a function to run ... the slow way

In [ ]:
def run_a_well_flopy(layer, cellid, newq, sfr_base_obs, depletion_results):
    # read in the simulation
    sim = fp.mf6.MFSimulation.load(sim_ws=str(modelpath))
    # get the model
    testmod = sim.get_model()
    # add the test well 
    well_cellid = (layer,cellid)
    twell = fp.mf6.ModflowGwfwel(testmod, save_flows=True, stress_period_data=[[well_cellid, newq]])
    runpath = Path('../data/depletion/')
    sim.set_sim_path(str(runpath))
    sim.exe_name='mf6'
    sim.write_simulation()
    sim.run_simulation()
    sfr_test = pd.read_csv(runpath / 'sfr_obs.csv')[['GAGE1','GAGE2']]
    depletion_results.loc[well_cellid, 'Gage1'] = (sfr_base_obs.GAGE1.values-sfr_test.GAGE1.values)/newq
    depletion_results.loc[well_cellid, 'Gage2'] = (sfr_base_obs.GAGE2.values-sfr_test.GAGE2.values)/newq
    

In [ ]:
for clay in range(3):
    for cellid in cells_for_wells:
        run_a_well_flopy(clay, cellid, newq,sfr_base_obs, depletion_results)

In [ ]:
depletion_results

In [ ]:
depletion_results.loc[0]


In [ ]:
l1g1 = depletion_results.loc[0].Gage1.values
l1g1[l1g1<0] = np.nan
mm = fp.plot.PlotMapView(model= base_m)
cb = mm.plot_array(depletion_results.loc[0].Gage1.values)
mm.plot_grid(lw=0.5, color="0.5")
plt.colorbar(cb, ax=plt.gca());

In [ ]:
l1g2 = depletion_results.loc[0].Gage2.values
l1g2[l1g2<0] = np.nan
mm = fp.plot.PlotMapView(model= base_m)
cb = mm.plot_array(depletion_results.loc[0].Gage2.values)
mm.plot_grid(lw=0.5, color="0.5")
plt.colorbar(cb, ax=plt.gca());

In [ ]:
np.nanmin(depletion_results.loc[0].Gage1.values)

In [ ]:
import os
os.getcwd()

# now make a function to run ... the FAST way
### we will use flopy only to make the initial model and then manipulate and run only with MF6 files directly

### make a test directory to run in

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws=str(modelpath))
# get the model
testmod = sim.get_model()
# add the test well 
twell = fp.mf6.ModflowGwfwel(testmod, save_flows=True, stress_period_data=[[(0,0), newq]])
runpath = Path('../data/depletion/')
sim.set_sim_path(str(runpath))
sim.exe_name='mf6'
sim.write_simulation()

### navigate the wel file to see where to put the new cell ids 

In [ ]:
inwell = open(runpath / 'project_0.wel', 'r').readlines()

In [ ]:
well_template = []
for line in inwell:
    if str(int(newq)) not in line:
        well_template.append(line)
    else:
        well_template.append('<replace_me>\n')

In [ ]:
well_template = ' '.join(well_template)

In [ ]:
well_template.replace('<replace_me>',f'1 1 {newq:0.4f}')

### now we can run these by just rewriting the wel file directly and suppressing output - should be faster

In [ ]:
# set the current working directory so that we can change as needed to run MF6
cwd = os.getcwd()

In [ ]:

def run_a_well_just_mf6(layer, cellid, newq, sfr_base_obs, depletion_results_fast):
    print(f'running layer = {layer}, cellid = {cellid}\r', end='')
    with open(runpath / 'project_0.wel','w') as ofp:
        ofp.write(well_template.replace('<replace_me>',f'{layer+1:d} {cellid+1:d} {newq:0.4f}'))
    well_cellid = (layer, cellid)
    os.chdir(runpath)
    os.system('mf6 > nul')
    os.chdir(cwd)
    sfr_test = pd.read_csv(runpath / 'sfr_obs.csv')[['GAGE1','GAGE2']]
    depletion_results_fast.loc[well_cellid, 'Gage1'] = (sfr_base_obs.GAGE1.values-sfr_test.GAGE1.values)/newq
    depletion_results_fast.loc[well_cellid, 'Gage2'] = (sfr_base_obs.GAGE2.values-sfr_test.GAGE2.values)/newq

In [ ]:
depletion_results_fast = pd.DataFrame(index=pd.MultiIndex.from_tuples(cells_with_layers), 
                                      data = {'Gage1':np.nan, 'Gage2':np.nan})
depletion_results_fast
for lay in range(1):
    for cid in cells_for_wells:
        run_a_well_just_mf6(lay, cid, newq, sfr_base_obs, depletion_results_fast)

In [ ]:
depletion_results_fast

In [ ]:
l1g1 = depletion_results_fast.loc[0].Gage1.values
l1g1[l1g1<0] = np.nan
mm = fp.plot.PlotMapView(model= base_m)
cb = mm.plot_array(depletion_results_fast.loc[0].Gage1.values)
mm.plot_grid(lw=0.5, color="0.5")
plt.colorbar(cb, ax=plt.gca());


In [ ]:
l1g2 = depletion_results_fast.loc[0].Gage2.values
l1g2[l1g2<0] = np.nan
mm = fp.plot.PlotMapView(model= base_m)
cb = mm.plot_array(depletion_results_fast.loc[0].Gage2.values)
mm.plot_grid(lw=0.5, color="0.5")
plt.colorbar(cb, ax=plt.gca());
